## MỞ ĐẦU 

Như chúng ta đã biết SVM là một trong hai phương pháp phân loại phổ biến nó phân loại dự trên hàm hinge loss.  Và phương pháp phân loại còn lại là Softmax. Điểm khác biệt giữa Softmax và SVM đó là hàm mất mát khác nhau. Nếu như hàm mất mát của SVM đánh giá mức độ hài lòng về chấm điểm phù hợp với class từ một điểm dữ liệu. Còn Soft max thì nó cụ thể hơn đó là xác xuất được chuẩn hóa tức là xác xuất của dữ liệu i vào lớp thứ j là bao nhiêu phần trăm. Nhìn chung ta có cách nhìn trực quan hơn. Bởi lẽ chúng ta đều hỏi xác xuất nhưng về điểm chúng ta khá mơ hồ về chúng.

Đối với SVM hàm số $f(x_i; W) = W x_i$ là điểm số, còn với Soft max sẽ là xác xuất. Và thay thế cho hàm hinge loss của SVM, Softmax cung cấp một hàm mất mát cross-entropy loss có dạng :

$L_i = -\log\left(\frac{e^{f_{y_i}}}{ \sum_j e^{f_j} }\right) \hspace{0.5in} \text{or equivalently} \hspace{0.5in} L_i = -f_{y_i} + \log\sum_j e^{f_j}$

Softmax sẽ lấy một giá trị của một vector, sau đó nén chúng lại dạng xác xuất trong đoạn [0,1] và tổng xác xuất của điểm dữ liệu đó trong các class bằng 1.

Vấn đề thực tế : Trong thực tế khi bạn viết mã cho việc tính toán hàm Softmax các phép toán trung gian $e^{f_{y_i}}$ và $\sum_j e^{f_j}$ có thể rất lớn do hàm mũ, điều đó khiến dữ liệu không ổn định vì vậy chúng ta cần thực hiện thao tác chuẩn hóa dữ liệu nếu chúng ta nhân đỉnh và đáy của phân số với một hằng số C và đẩy nó vào tổng, chúng ta sẽ nhận được biểu thức (tương đương về mặt toán học) sau đây:

$\frac{e^{f_{y_i}}}{\sum_j e^{f_j}}
= \frac{Ce^{f_{y_i}}}{C\sum_j e^{f_j}}
= \frac{e^{f_{y_i} + \log C}}{\sum_j e^{f_j + \log C}}$

Một cách lựa chọn phổ biến chúng ta thường sử dụng đó là lựa chọn $\log C = -\max_j f_j$

## ÁP DỤNG VỚI THƯ VIỆN SKLEARN GIẢI QUYẾT BÀI TOÁN BỘ DỮ LIỆU CIFA-10

Để sử dụng Softmax Regression, ta cần thêm một vài thuộc tính nữa:
linear_model.LogisticRegression(C=1e5, solver = 'lbfgs', multi_class = 'multinomial')
Với Logistic Regression, multi_class = 'ovr' là giá trị mặc định, tương ứng với one-vs-rest. solver = 'lbfgs' là một phương pháp tối ưu cũng dựa trên gradient nhưng hiệu quả hơn và phức tạp hơn Gradient Descent.

In [2]:
import pickle 
import numpy as np
import os
from scipy.misc import imread
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        datadict = pickle.load(fo, encoding='bytes')
    
    Y = datadict[b'labels']
    X = datadict[b'data']
    
    X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
    Y = np.array(Y)
    return X, Y
def load_CIFAR10(ROOT):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = unpickle(f)
        xs.append(X)
        ys.append(Y)    
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = unpickle(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

Thực hiện import thư viện và load dữ liệu và các mảng sau đó tiến hành chuyển các mảng đó sang ma trận để có thể dễ dàng thực hiện thuật toán.

In [3]:
cifar10_dir = 'F:\DataML\cifar-10-python\cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print ('Training data shape: ', X_train.shape)
print ('Training labels shape: ', y_train.shape)
print ('Test data shape: ', X_test.shape)
print ('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [4]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print ('Training data shape: ', X_train.shape)
print ('Training labels shape: ', y_train.shape)
print ('Test data shape: ', X_test.shape)
print ('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 3072)
Training labels shape:  (50000,)
Test data shape:  (10000, 3072)
Test labels shape:  (10000,)


Chúng ta có dữ liệu 2 loại là train và test rồi bây giờ chỉ cần cho hàm chạy thực hiện mô hình.

In [6]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(C = 1e5,solver = 'lbfgs', multi_class = 'multinomial').fit(X_train, y_train)
# Dự đoán mô hình và đánh giá kết quả.

y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score
print ("Hiệu quả mô hình theo Softmax là : %.2f %%" %(100*accuracy_score(y_test, y_pred.tolist())) )

Hiệu quả mô hình theo Softmax là : 40.42 %


C:\Users\trung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Khá tốt đấy chứ nhỉ trọng khi chúng ta không cần phải làm gì nhiều thì đã được kết quả thế này. 
So sánh với SVM xem sao nhỉ.

In [6]:
np.asanyarray(y_train)

mean = np.mean(X_train, axis = 0)
X_train = X_train - mean
X_test  = X_test - mean
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=1000)


In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf', degree = 3, gamma=1, C = 100)
clf.fit(X_train1, y_train1)
# Dự đoán mô hình và đánh giá kết quả.

y_pred = clf.predict(X_test1)

from sklearn.metrics import accuracy_score
print ("Hiệu quả mô hình theo SVM là : %.2f %%" %(100*accuracy_score(y_test, y_pred.tolist())) )